In [8]:
from sklearn.linear_model import Ridge
import joblib
import numpy as np
from sklearn.metrics import accuracy_score
from quadratic_weighted_kappa import quadratic_weighted_kappa
import pandas as pd

In [9]:
x = joblib.load('essay_ease10_sbert768_simbow_langerr_780_normalized_asap1')
asap1 = joblib.load("asap1_new_adjudication")
x.shape

(1783, 780)

In [10]:
asap1.head()

,essay,rater1_domain1,rater2_domain1,domain1_score,higher,mean,sum
0,"Dear local newspaper, I think effects computer...",4,4,8,4,4.0,8
1,"Dear @CAPS1 @CAPS2, I believe that using compu...",5,4,9,5,5.0,9
2,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...",4,3,7,4,4.0,7
3,"Dear Local Newspaper, @CAPS1 I have found that...",5,5,10,5,5.0,10
4,"Dear @LOCATION1, I know having computers has a...",4,4,8,4,4.0,8


In [83]:
y = asap1["sum"]

In [52]:
def get_feature_names_extended():
    ease_feats = ['Answer Length', 'Word Counts', 'Average Word Length', 'Good n-gram', 'Prompt Overlap',
              'Prompt Overlap (synonyms)', 'Punctuation Counts', 'Spelling Error', 'Unique Words', 'Prompt Similarity SBert']

    sbert_feats = []
    sbert_dim = 768

    for i in range(0, sbert_dim):
    	fname = "sbert_" + str(i) 
    	sbert_feats.append(fname)
    
    prompt_similarity_bow = ["Prompt Similarity BOW"]
    lang_error = ["Language Error"]
    
    feature_names = ease_feats + prompt_similarity_bow + lang_error + sbert_feats 

    print("len feature names: ", len(feature_names))
    
    return feature_names

feature_names = get_feature_names_extended()[:12]

len feature names:  780


In [84]:
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold

kf = KFold(n_splits=5, shuffle=True, random_state=42)

print(kf)

KFold(n_splits=5, random_state=42, shuffle=True)


In [97]:
model2 = Ridge(alpha=10)

In [98]:
qwk_scores = []

test_indices = []
pred_labels = []

for train_index, test_index in kf.split(x, y):
#     print("train_index: ", len(train_index))
#     print("test_index: ", len(test_index))
#     print("train_index: ", train_index[:20])
#     print("test_index: ", test_index[:20])
    
    X_train, X_test, Y_train, Y_test = x[train_index], x[test_index], y[train_index], y[test_index]

    #d_train = xgboost.DMatrix(X_train, label=Y_train, feature_names=feature_names)
    #d_test = xgboost.DMatrix(X_test, label=Y_test, feature_names=feature_names)
         
    model2.fit(X_train, Y_train)
    
    #rf.fit(X_train, Y_train)
    
    #predict = model.predict(d_test)
    predict = model2.predict(X_test)
    #predict = rf.predict(X_test)
    predict = np.round(predict)
    
    pred_labels.extend(predict)
    test_indices.extend(test_index)

    result = quadratic_weighted_kappa(Y_test, predict)
    
    print("Qwk : ", result)
    
    qwk_scores.append(result)
    
    
np.mean(qwk_scores)

Qwk :  0.729774459241013
Qwk :  0.7105300949809662
Qwk :  0.7323405523196371
Qwk :  0.7551357300073367
Qwk :  0.7601162749699323


0.737579422303777

In [99]:
print(len(pred_labels))
print(len(test_indices))

1783
1783


In [100]:
pred_labels_int = list(map(int, pred_labels))

In [101]:
new_score = np.zeros(len(pred_labels))
new_score

array([0., 0., 0., ..., 0., 0., 0.])

In [102]:
new_score[test_indices] = pred_labels_int

In [103]:
joblib.dump(new_score, 'score_model_normalized_asap1_rr_sum')

['score_model_normalized_asap1_rr_sum']